# Marching tetrahedra:  Separating the vertices of a cube using interpolating triangles

## Marching tetrahedra converts a 3d array of numbers like this:

In [1]:
import numpy as np
A = np.array([
    [
        [0, 1, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    [
        [0, 0, 0],
        [0, 1, 0],
        [0, 0, 0],
    ],
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
    [
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
    ],
])
A.shape

(4, 3, 3)

## Into a bunch of triangles defining an iso-surface like this:

In [2]:
from feedWebGL2.volume import display_isosurface, widen_notebook
widen_notebook()

simple_diagram = display_isosurface(A, threshold=0.3, save=True)

Volume32(status='deferring flush until render')

## There is a more complicated method which does not use tetrahedra: Marching cubes.

## That method has too many branches for GPU implementation.

## Branches are bad -- they make the processors wait and do nothing.


# The method produces triangle vertices and triangle normal vectors as output (used for coloration)

In [3]:
simple_diagram.doodle_diagram(all_corners=False)

DualCanvasWidget(status='deferring flush until render')


## Instead of triangulating cubes directly we break the cube into 6 tetrahedra and triangulate the tetrahedra

In [4]:
import feedback_diagrams

feedback_diagrams.tetrahedral_tiling()

DualCanvasWidget(status='deferring flush until render')

## Then in each tetrahedron:

## Separate vertices above the threshold from vertices below the threshold either use 2 triangles

In [5]:
feedback_diagrams.triangulated()

DualCanvasWidget(status='deferring flush until render')

## Or 1 triangle

In [6]:
feedback_diagrams.triangulated1()

DualCanvasWidget(status='deferring flush until render')

# In pseudo-Python the triangles are generated from crossing voxels like this:

```Python

# Per instance = voxel
for corners in crossing_voxel_corners:  # in parallel
    
    # Per mesh = 6 tetrahedra with up to 2 triangles each
    for tetrahedron in tetrahedra6(corners):   # parallelized using array indexing
        for triangle in triangles2(tetrahedron):   # parallelized using array indexing
            if triangle.is_valid():
                yield triangle
            else:
                yield None  # yield degenerate triangle if not valid
                # Every iteration must yield something on the GPU...
```

# The detailed implementation involves arcane use of array index lookups to avoid branching (branches are bad)

# The high level pipeline looks like this:

## -- Identify "crossing voxels" which cross the threshold and therefore have a peice of the isosurface
## -- Divide the crossing voxels into 6 tetrahedra
## -- Generate 0, 1, or 2 (non-degenerate) triangles from each tetrahedron
## -- Pass the generated triangles and the triangle normal vectors to the next pipeline stage (usually in three.js)

# Note it is possible to tile a voxel with just 5 tetrahedra

## But it would make the algorithm more complex because the line segments don't line up on opposite sides

In [7]:
feedback_diagrams.alt_tiling()

DualCanvasWidget(status='deferring flush until render')